In [ ]:
from glob import glob
import jaydebeapi
import numpy
import pandas

ModuleNotFoundError: No module named 'jaydebeapi'

<h3>環境設置</h3>

In [2]:
datatable='INFO'
field_names=['S_ID','G_ID','AMOUNT','PRICE']

<h3>開啟資料庫</h3>

In [25]:
dbConnection = jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:C:/Users/user/anaconda3/Projects/data/H2",
    ["SA", ""],
    "C:/Java/h2/bin/h2-1.4.200.jar")

dbCursor = dbConnection.cursor()

<h3>查詢數量</h3>

In [7]:
def search(gid):
    rows={
        'G_ID':[],
        'AMOUNT':[]
     }
    _sql= "SELECT * from INFO WHERE G_ID= '%s'" % gid 
    dbCursor.execute(''.join(_sql))
    resultSet = dbCursor.fetchall() #tuple資料
    
    for row in resultSet:
        amount=0
        id=gid
        amount+=row[2] 
        #rows['G_ID']=id
        #rows['AMOUNT']=amount 

    #results=pandas.DataFrame(data=rows).set_index(['G_ID']) #把row指派到data裡面
    return ("id=",id,"amount=",amount)

gid='1'
rows={
    'G_ID':[],
    'AMOUNT':[]
 }
_sql= "SELECT * from INFO WHERE G_ID= '%s'" % gid 
dbCursor.execute(''.join(_sql))
resultSet = dbCursor.fetchall() #tuple資料
amount=0
for row in resultSet:
    amount+=row[2] 
    rows['G_ID'].append(row[1])
rows['AMOUNT']=amount 

results=pandas.DataFrame(data=rows).set_index(['G_ID']) #把row指派到data裡面
results

In [8]:
search(8)

('id=', 8, 'amount=', 590)

<h3>查詢庫存最低前五名</h3>

In [30]:
rows={
    'G_ID':[],
    'AMOUNT':[]
 }

for gid in range(15):
    _sql= "SELECT * from INFO WHERE G_ID= '%s'" % gid 
    dbCursor.execute(''.join(_sql))
    resultSet = dbCursor.fetchall() #tuple資料
     
    for row in resultSet:
        amount=0
        amount+=row[2] 
    rows['AMOUNT'].append(row[2])
    rows['G_ID'].append(gid)
    
    _results=pandas.DataFrame(data=rows).set_index(['G_ID']) #把row指派到data裡面

a_results=_results.sort_values(by='AMOUNT')
a_results.head(5)

AMOUNT
G_ID        
7     -33589
5     -33285
9     -33209
8     -33178
4     -33085

<h3>販賣交易資料檔函式</h3>

In [18]:
t_results=[]
def tran(amount,sid,gid):
    rows={
        'S_ID':[],
        'G_ID':[],
        'AMOUNT':[],
        'PRICE':[]
    }
    _sql= "SELECT S_ID,G_ID,AMOUNT,PRICE, AMOUNT-'%d' AS NEW_AMOUNT from INFO WHERE S_ID='%s'AND G_ID='%s'" %(amount,sid,gid) #多一行(5)放新的數量
    dbCursor.execute(''.join(_sql))
    resultSet = dbCursor.fetchall() #tuple資料
    #resultSet[0][4]最新交易資料的數量結果
    
    #資料結構(陣列),初始化
    for row in resultSet:
        rows['S_ID'].append(row[0])
        rows['G_ID'].append(row[1])
        rows['AMOUNT'].append(row[4]) #將舊資料用新資料蓋過
        rows['PRICE'].append(row[3])
    
    _results=pandas.DataFrame(data=rows).set_index(['S_ID','G_ID']) #把row指派到data裡面
    _update_table_sql="UPDATE %s SET %s=%d WHERE S_ID='%s' AND G_ID='%s'" %(datatable,'AMOUNT',resultSet[0][4],sid,gid)
    dbCursor.execute(''.join(_update_table_sql))
    t_results.append(_results)
    return t_results

In [32]:
tran(10,1,4)
tran(10,0,4)
tran(10,0,5)

[           AMOUNT  PRICE
 S_ID G_ID               
 1    5     -33285    380,
            AMOUNT  PRICE
 S_ID G_ID               
 1    4     -32913    501,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4     -33095    117,
            AMOUNT  PRICE
 S_ID G_ID               
 0    5     -32857    830,
            AMOUNT  PRICE
 S_ID G_ID               
 1    4     -32923    501,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4     -33105    117,
            AMOUNT  PRICE
 S_ID G_ID               
 0    5     -32867    830]

<h3>說明</h3>
amount=10
sid=1
gid=5
t_results=[]
rows={
        'S_ID':[],
        'G_ID':[],
        'AMOUNT':[],
        'PRICE':[]
}
_sql= "SELECT S_ID,G_ID,AMOUNT,PRICE, AMOUNT-'%d' AS NEW_AMOUNT from INFO WHERE S_ID='%s'AND G_ID='%s'" %(amount,sid,gid) #多一行(5)放新的數量
dbCursor.execute(''.join(_sql))
resultSet = dbCursor.fetchall() #tuple資料

    
#資料結構(陣列),初始化
for row in resultSet:
    rows['S_ID'].append(row[0])
    rows['G_ID'].append(row[1])
    rows['AMOUNT'].append(row[4]) #將舊資料用新資料蓋過
    rows['PRICE'].append(row[3])
    
_results=pandas.DataFrame(data=rows).set_index(['S_ID','G_ID']) #把row指派到data裡面
_update_table_sql="UPDATE %s SET %s=%d WHERE S_ID='%s' AND G_ID='%s'" %(datatable,'AMOUNT',resultSet[0][4],sid,gid)
dbCursor.execute(''.join(_update_table_sql))
t_results.append(_results)
resultSet

<h3>補貨資料檔函式</h3>

In [12]:
p_results=[]
def purchase(amount,sid,gid):
    rows={
        'S_ID':[],
        'G_ID':[],
        'AMOUNT':[],
        'PRICE':[]
    }
    _sql= "SELECT S_ID,G_ID,AMOUNT,PRICE, AMOUNT+'%d' AS NEW_AMOUNT from INFO WHERE S_ID='%s'AND G_ID='%s'" %(amount,sid,gid) #多一行(5)放新的數量
    dbCursor.execute(''.join(_sql))
    resultSet = dbCursor.fetchall() #tuple資料
    #resultSet[0][4]
    #資料結構(陣列),初始化

    for row in resultSet:
        rows['S_ID'].append(row[0])
        rows['G_ID'].append(row[1])
        rows['AMOUNT'].append(row[4]) #將舊資料用新資料蓋過
        rows['PRICE'].append(row[3])

    _results=pandas.DataFrame(data=rows).set_index(['S_ID','G_ID']) #把row指派到data裡面
    _update_table_sql="UPDATE %s SET %s=%d WHERE S_ID='%s' AND G_ID='%s'" %(datatable,'AMOUNT',resultSet[0][4],sid,gid)
    dbCursor.execute(''.join(_update_table_sql))
    p_results.append(_results)
    return p_results

<h3>測試</h3>

In [13]:
purchase(10,1,4)
purchase(10,0,4)
purchase(10,0,5)

[           AMOUNT  PRICE
 S_ID G_ID               
 1    4        637    501,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4        465    117,
            AMOUNT  PRICE
 S_ID G_ID               
 0    5        790    830]

<h3>測試</h3>

In [14]:
for i in range(1000):
    tran(i+1,0,i%15)
    tran(i+1,1,i%15)  
tran(10,0,4)

[           AMOUNT  PRICE
 S_ID G_ID               
 1    4        627    501,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4        455    117,
            AMOUNT  PRICE
 S_ID G_ID               
 0    5        780    830,
            AMOUNT  PRICE
 S_ID G_ID               
 0    0        714    854,
            AMOUNT  PRICE
 S_ID G_ID               
 1    0        580    597,
            AMOUNT  PRICE
 S_ID G_ID               
 0    1        920    500,
            AMOUNT  PRICE
 S_ID G_ID               
 1    1        750    696,
            AMOUNT  PRICE
 S_ID G_ID               
 0    2        419    925,
            AMOUNT  PRICE
 S_ID G_ID               
 1    2        595    655,
            AMOUNT  PRICE
 S_ID G_ID               
 0    3        336    698,
            AMOUNT  PRICE
 S_ID G_ID               
 1    3        602    242,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4        460    117,
            AMOUNT  PRICE
 S_ID G_ID               


In [16]:
for i in range(15):
    purchase(i+1,0,i)
    purchase(i+1,1,i) 
purchase(10,0,5)

[           AMOUNT  PRICE
 S_ID G_ID               
 1    4        184    606,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4        933    955,
            AMOUNT  PRICE
 S_ID G_ID               
 0    5        725    924,
            AMOUNT  PRICE
 S_ID G_ID               
 0    0     -66004    156,
            AMOUNT  PRICE
 S_ID G_ID               
 1    0     -65905    189,
            AMOUNT  PRICE
 S_ID G_ID               
 0    1     -65832    256,
            AMOUNT  PRICE
 S_ID G_ID               
 1    1     -66193    833,
            AMOUNT  PRICE
 S_ID G_ID               
 0    2     -66715     88,
            AMOUNT  PRICE
 S_ID G_ID               
 1    2     -66155    828,
            AMOUNT  PRICE
 S_ID G_ID               
 0    3     -66001    259,
            AMOUNT  PRICE
 S_ID G_ID               
 1    3     -66123    458,
            AMOUNT  PRICE
 S_ID G_ID               
 0    4     -66082    955,
            AMOUNT  PRICE
 S_ID G_ID               


In [22]:
dbCursor.close()
dbConnection.close()

In [1]:
import pandas as pd
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1, 2, 3, 10, 20, 30] )
s

0     1
1     2
2     3
3    10
4    20
5    30
dtype: int64